<a href="https://colab.research.google.com/github/singr7/blip-image-captioning/blob/main/ImageCaptionGradioApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
PORT1=57904

In [10]:
hf_api="your_api_key_here"

secrets_dir='/content/gdrive/My Drive/secrets.env'

In [23]:
!# hf_api variable should have your actual Hugging Face API key
!echo "HF_API_KEY={hf_api}" > "/content/gdrive/My Drive/secrets.env"


#Load your Hugging face API key

In [ ]:
!pip install python-dotenv
!pip install transformers

In [ ]:
!pip install gradio

In [14]:
import os
import io
import IPython.display
from PIL import Image
import base64


In [24]:
from google.colab import drive

drive.mount('/content/gdrive')

# Path to the secrets.env file in Google Drive
secrets_file_path = '/content/gdrive/My Drive/secrets.env'

# Read the secrets.env file and set the environment variables
with open(secrets_file_path, 'r') as f:
    for line in f:
        key, value = line.strip().split('=')
        os.environ[key] = value

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
hf_api_key = os.environ['HF_API_KEY']

In [29]:
# Helper functions
import requests, json

#Image-to-text endpoint
def get_completion(inputs, parameters=None, ENDPOINT_URL=endpoint_url ):
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL,
                                headers=headers,
                                data=json.dumps(data))
    return json.loads(response.content.decode("utf-8"))

In [31]:
#Alternative to running it from the hosted inference API - So in local
from transformers import pipeline

get_completion = pipeline("image-to-text",model="Salesforce/blip-image-captioning-base")

def summarize(input):
    output = get_completion(input)
    return output[0]['generated_text']

In [39]:
image_url = "https://free-images.com/sm/9596/dog_animal_greyhound_983023.jpg"
display(IPython.display.Image(url=image_url))
get_completion(image_url)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'a dog wearing a santa hat and a red scarf'}]

In [44]:
import gradio as gr

def image_to_base64_str(pil_image):
    byte_arr = io.BytesIO()
    pil_image.save(byte_arr, format='PNG')
    byte_arr = byte_arr.getvalue()
    return str(base64.b64encode(byte_arr).decode('utf-8'))

def captioner(image):
    base64_image = image_to_base64_str(image)
    result = get_completion(base64_image)
    return result[0]['generated_text']

gr.close_all()
demo = gr.Interface(fn=captioner,
                    inputs=[gr.Image(label="Upload image", type="pil")],
                    outputs=[gr.Textbox(label="Caption")],
                    title="Image Captioning with BLIP",
                    description="Caption any image using the BLIP model",
                    allow_flagging="never", examples=None)
                    #examples= ["christmas_dog.jpeg", "bird_flight.jpeg", "cow.jpeg"])

demo.launch(share=True, server_port=PORT1)

Closing server running on port: 57904
Closing server running on port: 57904
Closing server running on port: 57904
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://11933a66ee74491164.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
gr.close_all()